### Inférence préparation

In [1]:
import cv2
import torch
import torchvision
import torch
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera


CATEGORIES = ['Train']
device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('RF1.pth'))
car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=65)

### Correcteur Proportionnel

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = -4.00
#STEERING_BIAS = -0.15
STEERING_BIAS = 0

car.throttle = -0.4

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS

In [ ]:
from utils import preprocess
import numpy as np

# Constantes pour le contrôle de la voiture
STEERING_GAIN = -2.00
STEERING_BIAS = 0

# Configuration initiale de la voiture
car.throttle = -0.3

try:
    while True:
        # Lire l'image depuis la caméra
        image = camera.read()
        
        # Prétraiter l'image
        image = preprocess(image).half()
        
        # Faire une prédiction avec le modèle
        output = model_trt(image).detach().cpu().numpy().flatten()
        
        # Calculer la commande de direction
        x = float(output[0])
        car.steering = x * STEERING_GAIN + STEERING_BIAS
except KeyboardInterrupt:
    print("Arrêt du programme par l'utilisateur.")
except Exception as e:
    print(f"Une erreur s'est produite : {e}")


### Correcteur Proportionnel et Dérivateur

In [ ]:
from utils import preprocess
import numpy as np

kp = -2.0
dd = 1
last_x = 0

car.throttle = -0.2
car.steering_offset = 0

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = (x * kp + ( x - last_x ) *dd)
    last_x = x

### Enregistrement erreur

In [ ]:
import csv
from utils import preprocess
import numpy as np

STEERING_GAIN = -2.00
STEERING_BIAS = 0

car.throttle = -0.3

# Ouvrir le fichier CSV en mode écriture
with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['x']  # Nom du champ dans le fichier CSV
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Écrire l'en-tête du fichier CSV
    writer.writeheader()

    while True:
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        car.steering = x * STEERING_GAIN + STEERING_BIAS
        
        # Écrire la valeur de x dans le fichier CSV
        writer.writerow({'x': x})
